## NLCD RASTERS
Extract tree canopy and impervious percentages.

In [1]:
# Modules.
import os
from pathlib import Path
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
import geopandas as gpd
from tqdm import tqdm
from rasterstats import zonal_stats

In [2]:
# Paths.
nlcd_tree_path = Path("data/raster/nlcd_raster/nlcd_tree_canopy_2023.tiff")
nlcd_impervious_path = Path("data/raster/nyc_impervious_2024.tif")

tracts_path = Path("data/nyc_tracts_2020/nyc_tracts_2020.shp")

output_dir = Path("data/raster/processed")
output_dir.mkdir(parents = True, exist_ok = True)

tracts = gpd.read_file(tracts_path).to_crs(32118)

In [3]:
# Zonal statistics for NCLD.
def zonal_mean(rpath, gdf_or_geom):
    """Apply CRS zonal mean for tracts or city boundary."""
    with rasterio.open(rpath) as src:
        r_crs = src.crs

    if isinstance(gdf_or_geom, (gpd.GeoSeries, gpd.GeoDataFrame)):
        gdf = gdf_or_geom.to_crs(r_crs)
    else:
        gdf = gpd.GeoSeries([gdf_or_geom], crs = 32118).to_crs(r_crs)

    return zonal_stats(gdf, rpath, stats = ["mean"], nodata = np.nan)

In [6]:
# Print checks for the calculations.
print("Tree canopy zonal stats:")
tree = zonal_mean(nlcd_tree_path, tracts)
print(tree)
print("Impervious zonal stats:")
impervious = zonal_mean(nlcd_impervious_path, tracts)
print(impervious)

tracts["pct_tree_canopy"] = [t["mean"] for t in tree]
tracts["pct_impervious"] = [i["mean"] for i in impervious]

Tree canopy zonal stats:
[{'mean': 10.126984126984127}, {'mean': 13.314285714285715}, {'mean': 0.592}, {'mean': 3.076305220883534}, {'mean': 6.089887640449438}, {'mean': 2.736842105263158}, {'mean': 1.7009803921568627}, {'mean': 1.2598870056497176}, {'mean': 6.25}, {'mean': 2.8217821782178216}, {'mean': 0.4077669902912621}, {'mean': 1.0520833333333333}, {'mean': 0.047619047619047616}, {'mean': 1.2023809523809523}, {'mean': 0.0}, {'mean': 7.0647482014388485}, {'mean': 0.07936507936507936}, {'mean': 0.05154639175257732}, {'mean': 1.3650793650793651}, {'mean': 0.15025906735751296}, {'mean': 6.22}, {'mean': 0.6020408163265306}, {'mean': 0.39896373056994816}, {'mean': 0.6956521739130435}, {'mean': 0.19270833333333334}, {'mean': 1.517766497461929}, {'mean': 4.198979591836735}, {'mean': 0.387434554973822}, {'mean': 0.10256410256410256}, {'mean': 0.05235602094240838}, {'mean': 0.8134715025906736}, {'mean': 4.455958549222798}, {'mean': 3.572972972972973}, {'mean': 0.0}, {'mean': 4.6783919597989

In [7]:
tracts.columns = tracts.columns.str.upper()
tracts = tracts.rename(columns = {"GEOMETRY": "geometry"})

tracts.head()

,CTLABEL,BOROCODE,BORONAME,CT2020,BOROCT2020,CDELIGIBIL,NTANAME,NTA2020,CDTA2020,CDTANAME,GEOID,SHAPE_LENG,SHAPE_AREA,geometry,PCT_TREE_CANOPY,PCT_IMPERVIOUS
0,1,1,Manhattan,000100,1000100,I,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,10833.043929,1.843005e+06,"MULTIPOLYGON (((296291.11 58135.714, 296322.49...",10.126984,46.973545
1,14.01,1,Manhattan,001401,1001401,I,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,5075.332000,1.006117e+06,"POLYGON ((300982.972 61050.77, 301053.207 6102...",13.314286,69.590476
2,14.02,1,Manhattan,001402,1001402,E,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4459.156019,1.226206e+06,"POLYGON ((301261.149 61343.714, 301332.269 613...",0.592000,83.656000
3,18,1,Manhattan,001800,1001800,I,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001800,6391.921174,2.399277e+06,"POLYGON ((300857.167 61503.238, 300868.535 614...",3.076305,81.887550
4,22.01,1,Manhattan,002201,1002201,E,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061002201,5779.062607,1.740174e+06,"POLYGON ((301795.202 61438.262, 301792.173 614...",6.089888,78.735955


In [8]:
tracts.columns

Index(['CTLABEL', 'BOROCODE', 'BORONAME', 'CT2020', 'BOROCT2020', 'CDELIGIBIL',
       'NTANAME', 'NTA2020', 'CDTA2020', 'CDTANAME', 'GEOID', 'SHAPE_LENG',
       'SHAPE_AREA', 'geometry', 'PCT_TREE_CANOPY', 'PCT_IMPERVIOUS'],
      dtype='object')

In [9]:
tracts = tracts.drop(columns = ['CTLABEL', 'BOROCODE', 'BORONAME', 'CT2020',
                       'BOROCT2020', 'CDELIGIBIL', 'NTANAME', 'NTA2020',
                       'CDTA2020', 'CDTANAME', 'SHAPE_LENG', 'SHAPE_AREA'])

In [10]:
tracts.columns

Index(['GEOID', 'geometry', 'PCT_TREE_CANOPY', 'PCT_IMPERVIOUS'], dtype='object')

In [11]:
# Save as geojson.
geojson_out = output_dir.parent / "nlcd_calc_tracts.geojson"
geojson_out.parent.mkdir(parents = True, exist_ok = True)
tracts.to_file(geojson_out)
print("Saved:", geojson_out)

# Save as csv.
csv_out = output_dir.parent / "nlcd_calc_tracts.csv"
tracts.drop(columns = ["geometry"]).to_csv(csv_out, index = False)
print("Saved CSV:", csv_out)

Saved: data\raster\nlcd_calc_tracts.geojson
Saved CSV: data\raster\nlcd_calc_tracts.csv
